In [ ]:
import nbimporter
import crypto_package as CP
import uniform_normal_probability_distributions as ND
import os, pickle, quandl, time, math
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib notebook
from PIL import Image


py.init_notebook_mode(connected = True)



########################################################
# compile data
########################################################

# exchange info
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','XMR','XEM', 'GNT']
alt_data = {}

#change dir to poloniex_cache & create dir if it doesn't exist 
if not 'poloniex_cache' in os.listdir(os.getcwd()):
    os.mkdir('poloniex_cache')
    
path = os.path.join(os.getcwd(), 'poloniex_cache')
os.chdir(path)

for alt in altcoins:
    coinpair = 'BTC_{}'.format(alt)
    crypto_price_df = CP.get_crypto_data(coinpair)
    alt_data[alt] = crypto_price_df
###############################################################




###############################################################
#leave the directory
os.chdir('../')

exchanges = ['KRAKEN','COINBASE', 'BITSTAMP','ITBIT','OKCOIN', 'GETBTC']#,'COINSBANK','HITBTC','LYBIT','ANXHK','BITME','BITBOX','INTRSNG','BTCE','WEEX','JUST','CBX']
exch_data = {}
# retrieve exchange data and read into dictionary
for exchange in exchanges:
    exchange_df = CP.get_quandl_data('BCHARTS/{}USD'.format(exchange))
    #time.sleep(1000)
    exch_data[exchange] = exchange_df
    
# Merge BTC price data series' into single dataframe
btc_usd_datasets = CP.merge_dfs_on_column(list(exch_data.values()), list(exch_data.keys()), 'Weighted Price')
btc_usd_datasets.replace(0, np.nan, inplace = True) # remove 0 values
# Calc avg in new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis = 1)



# calculate USD Price as new col in each dataframe
for alt in alt_data.keys():
    alt_data[alt]['price_usd'] = alt_data[alt]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']
    

########################################################
# organize into daily returns
########################################################
dailyReturns = {}

for alt in alt_data.keys():
    dailyReturns[alt] = np.empty(len(alt_data[alt]['price_usd']) - 1)
    for i in range(len(alt_data[alt]['price_usd']) - 1):
        # calculate returns for day
        dayRet = (alt_data[alt]['price_usd'][i + 1] - alt_data[alt]['price_usd'][i]) / alt_data[alt]['price_usd'][i]
        #insert them into numpy array
        np.insert(dailyReturns[alt], i, dayRet)

        
#matrix to contain preprocessed pearson correlation values between currencies
pearsonValueMatrix = np.empty([len(alt_data.keys()), len(alt_data.keys())])
indexVector = []
i = 0
j = 0
for a1 in alt_data.keys():
    indexVector.append(i)
    for a2 in alt_data.keys():
        np.insert(pearsonValueMatrix, [], CP.pearson_coefficient(dailyReturns[a1], dailyReturns[a2]))
        j += 1
    i += 1

print(pearsonValueMatrix)
pearsonValueMatrix
#img = Image.fromarray(pearsonValueMatrix, 'RGB')
#img.show()